In [1]:
# Load LSTM network and generate text
import sys
import numpy
from numpy import array
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers import Dense, Dropout , BatchNormalization
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import TimeDistributed
from keras.utils import to_categorical
from keras import optimizers
from keras.layers import Reshape
from keras.models import load_model
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras import optimizers
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


# load training data label in text form
raw_train_label_text = open('train_label.txt', 'r', encoding='utf-8').read()
raw_test_label_text =  open('test_label.txt', 'r', encoding='utf-8').read()

# convert text to integer
test_x = []
train_x = []
for _ in raw_train_label_text.splitlines():
    val = int(_)
    train_x.append(val)
for _ in raw_test_label_text.splitlines():
    val = int(_)
    test_x.append(val)

# summarize the loaded data
train_count = len(train_x)
test_count = len(test_x)
print ("Total Train: ", train_count)
print ("Total Test: ", test_count)

Total Train:  11084
Total Test:  5225


In [2]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [3]:
def debug_print_count(data, string='NA'):
    unique, counts = np.unique(data, return_counts=True)
    print(string)
    print(dict(zip(unique, counts)))

In [4]:
# choose a number of time steps
n_steps = 1
n_features = 1
n_neurons = 50
n_batch = 1
n_outputs = 12

# split into samples
TrainX, trainy = split_sequence(train_x, n_steps)
TestX, testy = split_sequence(test_x, n_steps)

print(TrainX.shape, trainy.shape)
print(TestX.shape, testy.shape)

# reshape from [samples, timesteps] into [samples, timesteps, features]
TrainX = TrainX.reshape((TrainX.shape[0], n_steps, n_features))
TestX = TestX.reshape((TestX.shape[0], n_steps, n_features))

debug_print_count(trainy, '[b4]Train Y: ')
debug_print_count(testy, '[b4]Test Y: ')

# one hot encode y
trainy = to_categorical(trainy)
testy = to_categorical(testy)

# process to throw the first row
trainy = trainy[:,1:13]
testy = testy[:,1:13]
    
print(TrainX.shape, trainy.shape)
print(TestX.shape, testy.shape)

# define model
model = Sequential()
model.add(LSTM(n_neurons, input_shape=(n_steps, n_features)))
model.add(Dropout(0.5))
model.add(Dense(n_outputs, activation='softmax'))
opt = optimizers.Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt)
 
# fit model
#model.fit(TrainX, trainy, epochs=10, verbose=1, shuffle=False)
#_, accuracy = model.evaluate(TestX, testy, batch_size=10, verbose=1)
#print("Accuracy: {:.2f} %".format(accuracy*100))
# demonstrate prediction
# fit network
#for i in range(10):
model.fit(TrainX, trainy, epochs=10, batch_size=n_batch, verbose=0, shuffle=False)
model.reset_states()

count = 0
# online forecast
for i in range(len(TestX)):
    x, y = TestX[i], testy[i]
    x = x.reshape((1, n_steps, n_features))
    y_pred = model.predict(x, batch_size=1)
    #print(y_pred)
    y_predict = np.argmax(y_pred, axis=1)
    y = np.argmax(testy[i])
    # add back the value to make sure activity label start from 1 instead of 0
    y_predict = y_predict + 1
    y = y + 1

    #print('>Expected=%.1f, Predicted=%.1f' % (y, y_predict))
    if (y == y_predict):
        count+= 1
#         print('>Expected=%.1f, Predicted=%.1f' % (y, y_predict))
#     else:
#          print('>Expected=%.1f, Predicted=%.1f' % (y, y_predict))
    #print("Accuracy: {:.2f} %".format(accuracy_score(testy, y_predict)*100))
    #print(confusion_matrix(testy, y_predict))
    #print(classification_report(testy, y_predict))
else:
    print("Correct guess:" + str(count))
    print("Accuracy = {:.2f} %".format((count/len(TestX))*100))

#print(confusion_matrix(testy, y_predict))
#print(classification_report(testy, y_predict))
#print("Accuracy: {:.2f} %".format(accuracy_score(testy, y_predict)*100))

(11083, 1) (11083,)
(5224, 1) (5224,)
[b4]Train Y: 
{1: 1704, 2: 1606, 3: 1471, 4: 1672, 5: 1855, 6: 1820, 7: 148, 8: 110, 9: 172, 10: 161, 11: 214, 12: 150}
[b4]Test Y: 
{1: 740, 2: 730, 3: 687, 4: 857, 5: 902, 6: 921, 7: 61, 8: 52, 9: 76, 10: 62, 11: 69, 12: 67}
(11083, 1, 1) (11083, 12)
(5224, 1, 1) (5224, 12)
Correct guess:4675
Accuracy = 89.49 %
